In [2]:
import openrouteservice as ors
import folium

client = ors.Client(key='5b3ce3597851110001cf6248f4b857eea0554d93886b4c09e748b2bf')

In [59]:
import re
import pandas as pd
import math

import numpy as np

from operator import itemgetter
from heapq import nsmallest

In [73]:
df_property = pd.read_csv('../data/curated/property.csv')

property_coor = []
for index, row in df_property.iterrows():
    if re.search(r'\[(-\d+.\d+)',str(row['coordinates'])): 
        lat = float(re.findall(r'\[(-\d+.\d+)',str(row['coordinates']))[0])
        long = float(re.findall(r'(\d+.\d+)\]',str(row['coordinates']))[0])
        property_coor.append((long,lat))
    else:
        property_coor.append((0,0))
print(len(property_coor))

df_school = pd.read_csv('../data/curated/school.csv')

school_coor = []
for index, row in df_school.iterrows():
    if not math.isnan(row['Y']):
        lat = row['Y']
        long = row['X']
        school_coor.append((long,lat))
    else:
        school_coor.append((0,0))
print(len(school_coor))
print(df_school['X'])


15226
2299
0       146.66660
1       142.59039
2       143.47565
3       145.23472
4       145.21398
          ...    
2294    145.31604
2295    145.08148
2296    144.90520
2297    145.31589
2298    145.29327
Name: X, Length: 2299, dtype: float64


In [78]:
# find the closest 4 schools for each property based on euclidean distance
# api has accessed limits so that it is difficult to track the route between each school and each property

from unicodedata import name


school_coor = np.array(school_coor)
property_coor = np.array(property_coor)

def get_school(i):
    # calculate euclidean distance
    distances = np.linalg.norm(school_coor-property_coor[i], axis=1)
    min_index = np.argmin(distances)

    # find the 4 nearest school
    index = list(range(len(school_coor)))
    pairs = zip(index, distances)
    result = nsmallest(4, pairs, key=itemgetter(1))
    index_list = [i[0] for i in result]

    # store the name and the coordinates of the 4 nearest schools
    name_list = [df_school['School_Name'][i] for i in index_list]
    coor_list = [(df_school['X'][i],df_school['Y'][i])for i in index_list]

    return name_list, coor_list

# for each property, get corresponding closest schools
name_list = []
coor_list = []

for i in range(len(property_coor)):
    name_list.append(get_school(i)[0])
    coor_list.append(get_school(i)[1])


In [46]:
combine_coor = property_coor + school_coor

print(len(combine_coor))

def get_closest_school(iter):

    matrix = client.distance_matrix(
        locations=combine_coor,
        profile='driving-car',
        metrics=['distance', 'duration'],
        validate=False,
        destinations=[15227,15228],
        sources=iter,
    )
    
    
    print("Durations in secs: {}\n".format(matrix['durations']))
    print("Distances in m: {}".format(matrix['distances']))

get_closest_school(0)

17525


ApiError: 400 ({'error': {'code': 6004, 'message': 'Request parameters exceed the server configuration limits. Only a total of 3500 routes are allowed.'}, 'info': {'engine': {'version': '6.7.0', 'build_date': '2022-02-18T19:37:41Z'}, 'timestamp': 1663562208504}})

In [55]:
import requests

l = range(15227, len(combine_coor))

body = {"locations":combine_coor,"destinations":list(l),"metrics":["distance","duration"],"sources":[0,1]}

headers = {
    'Accept': 'application/json, application/geo+json, application/gpx+xml, img/png; charset=utf-8',
    'Authorization': '5b3ce3597851110001cf6248f4b857eea0554d93886b4c09e748b2bf',
    'Content-Type': 'application/json; charset=utf-8'
}
call = requests.post('https://api.openrouteservice.org/v2/matrix/driving-car', json=body, headers=headers)

print(call.status_code, call.reason)
print(1)
print(call.text)

400 Bad Request
1
{"error":{"code":6004,"message":"Request parameters exceed the server configuration limits. Only a total of 3500 routes are allowed."},"info":{"engine":{"version":"6.7.0","build_date":"2022-02-18T19:37:41Z"},"timestamp":1663562618493}}
